In [1]:
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.2 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/UCF/BogenMaterials')
!ls


customer_queries.json  parsed_data.csv	   runs			text-sections
final_predictions.csv  parsed_data.gsheet  test.json		train.json
json_training_data     pdfs		   text_questions.json	val.json


In [3]:
from huggingface_hub import notebook_login

# Login to Hugging Face
notebook_login()

In [4]:
!pip install -U pip
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1
!pip install transformers==4.41.2  # Or the closest available version
!pip install datasets==2.20.0 accelerate==0.31.0 bitsandbytes==0.43.1
!pip install peft==0.11.1 trl==0.9.4 colored==2.2.4


from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import json
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

In [ ]:
MODEL_NAME = "josedellasala/Llama-3-8B-Instruct-E7000-Final-RAG"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto"
)
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=125,
    return_full_text=False,
)


# Step 2: Define model details
BASE_MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

# Load tokenizer and model
base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, use_fast=True)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto"
)
base_pipe = pipeline(
    task="text-generation",
    model=base_model,
    tokenizer=base_tokenizer,
    max_new_tokens=125,
    return_full_text=False,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
# Code to evaluate base model vs instructed model on Bogens Final Examination
with open("text_questions.json", "r") as file:
    questions = json.load(file)
def generate_prompt_and_answer(data_entry, pipeline):
    """
    Generates a prompt based on whether the question is multiple choice,
    multiple choice (select one or more), or completion, and gets an answer
    from the LLM pipeline.

    Args:
        data_entry (dict): A dictionary containing the question, options, type, and answer.
        pipeline (pipeline): The Hugging Face pipeline for text generation.

    Returns:
        dict: Contains the question, generated answer, and expected answer for comparison.
    """
    # Base prompt
    base_prompt = "Use your knowledge of Nyquist E7000 system to answer the following question beriefly and with no explanation:\n\n"

    num_tokens = 5

    # Check the question type
    if data_entry["type"] == "multiple choice":
        # Add the question and answer options for single-choice questions
        prompt = (
            base_prompt +
            f"Question: {data_entry['question']}\n" +
            "Options:\n" +
            "\n".join(data_entry["answer_options"]) +
            "\nAnswer (choose only one option):"
        )
    elif data_entry["type"] == "multiple choice (select one or more)":
        # Add the question and answer options for multi-choice questions
        prompt = (
            base_prompt +
            f"Question: {data_entry['question']}\n" +
            "Options:\n" +
            "\n".join(data_entry["answer_options"]) +
            "\nAnswer (choose more than one option, if applicable):"
        )
    elif data_entry["type"] == "completion":
        # Add the question only for completion-type questions
        prompt = (
            base_prompt +
            f"Question: {data_entry['question']}\n" +
            "Provide a concise answer:"
        )
        num_tokens = 15
    else:
        raise ValueError(
            f"Unknown question type '{data_entry['type']}'. Allowed types are: "
            "'multiple choice', 'multiple choice (select one or more)', 'completion'."
        )

    # Generate a concise answer
    generated_answer = pipeline(
        prompt,
        max_new_tokens=num_tokens,  # Allow up to 15 tokens for multi-choice answers
        temperature=0.2,   # Reduce randomness for deterministic answers
        top_p=0.9,         # Limit output to the most probable tokens
        return_full_text=False
    )[0]["generated_text"]

    # Return results
    return {
        "question": data_entry["question"],
        "generated_answer": generated_answer.strip(),
        "expected_answer": data_entry["answer"]
    }

# Example usage
# Assuming `questions` is the dataset loaded into a list of dictionaries
FTresults = []
BASEresults = []
for entry in questions:
    try:
        FTresults.append(generate_prompt_and_answer(entry, pipe))
        BASEresults.append(generate_prompt_and_answer(entry,base_pipe))
    except ValueError as e:
        print(f"Error processing question {entry.get('question_number', 'unknown')}: {str(e)}")

# Save the results to a file
with open("FTresults_generated_answers.json", "w") as outfile:
    json.dump(FTresults, outfile, indent=4)

with open("BASEresults_generated_answers.json", "w") as outfile:
    json.dump(BASEresults, outfile, indent=4)

# Print a summary
print("Generated answers saved.")

In [ ]:
import json
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch

# Load customer queries
with open("customer_queries.json", "r") as file:
    customer_queries = json.load(file)

# Function to generate a prompt
def generate_prompt(query):
    return (
        "You are a customer service representative expert in the Bogen E7000 system.\n"
        "Your task is to provide a possible explanation for the issue described below and suggest a possible fix.\n\n"
        f"Customer Query: {query}\n\n"
        "Explanation and Fix:"
    )

# Generate answers using both models
fine_tuned_responses = []
base_model_responses = []

for query in customer_queries:
    prompt = generate_prompt(query["query"])

    # Get responses from both models
    fine_tuned_answer = pipe(prompt, max_new_tokens = 125)[0]["generated_text"].strip()
    base_model_answer = base_pipe(prompt, max_new_tokens = 125)[0]["generated_text"].strip()

    # Append results
    fine_tuned_responses.append({
        "id": query["id"],
        "query": query["query"],
        "response": fine_tuned_answer
    })

    base_model_responses.append({
        "id": query["id"],
        "query": query["query"],
        "response": base_model_answer
    })

# Save results to JSON files
with open("fine_tuned_model_responses.json", "w") as fine_tuned_file:
    json.dump(fine_tuned_responses, fine_tuned_file, indent=4)

with open("base_model_responses.json", "w") as base_model_file:
    json.dump(base_model_responses, base_model_file, indent=4)

print("Responses saved to 'fine_tuned_model_responses.json' and 'base_model_responses.json'.")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Responses saved to 'fine_tuned_model_responses.json' and 'base_model_responses.json'.
